In [1]:
import cupy as cp 
import numpy as np
import pygraphblas as gb

# Check if connected to GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU: need to configure runtime type to an environment with an accessible GPU')
else:
    print('Connected to a GPU - GPU info summary: \n\n' + gpu_info)
    
# Use python wrapper of GraphBLAS on GPU (BLAS - Basic Linear Algebra Subprograms)
# GraphBLAS supports graph operations via linear algebraic methods (e.g. matrix multiplication) over various semirings

# GraphBLAS version of BMLP-RMS algorithm which performs repeated matrix squaring
from bmlp.matrix import *

Connected to a GPU - GPU info summary: 

Thu Nov 14 15:55:43 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.58.02              Driver Version: 556.12         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...    On  |   00000000:47:00.0  On |                  N/A |
| 30%   27C    P8             11W /  285W |     566MiB /  16376MiB |      9%      Default |
|                                         |                        |                  N/A |
+------

In [ ]:
# Create a sequence of boolean matrix operations to compute
#   p0(X1, X2) :- p1(X1, X3), p2(X3, X4), p3(X4, X2).
num_nodes = 5
p1 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)
p2 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)
p3 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)

p1[0,1] = True
print('p1 = \n' + str(p1) + '\n')

p2[1,2] = True
print('p2 = \n' + str(p2) + '\n')

p3[2,3] = True
print('p3 = \n' + str(p3) + '\n')

# exactly-two-connected program in chained H2m
p0 = p1 @ p2 @ p3
print('p0 = \n' + str(p0) + '\n')

p1 = 
      0  1  2  3  4
  0|     t         |  0
  1|               |  1
  2|               |  2
  3|               |  3
  4|               |  4
      0  1  2  3  4

p2 = 
      0  1  2  3  4
  0|               |  0
  1|        t      |  1
  2|               |  2
  3|               |  3
  4|               |  4
      0  1  2  3  4

p3 = 
      0  1  2  3  4
  0|               |  0
  1|               |  1
  2|           t   |  2
  3|               |  3
  4|               |  4
      0  1  2  3  4

p0 = 
      0  1  2  3  4
  0|           t   |  0
  1|               |  1
  2|               |  2
  3|               |  3
  4|               |  4
      0  1  2  3  4



In [3]:
# Create a sequence of boolean matrix operations to compute
#   p0(X1, X2) :- p1(X2, X1).
#   p0(X1, X2) :- p2(X1, X3), p3(X3, X4), p0(X4, X2).
num_nodes = 5
p1 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)
p2 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)
p3 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)

p1[1,0] = True
print('p1 = \n' + str(p1) + '\n')

p2[1,2] = True
p2[3,4] = True
print('p2 = \n' + str(p2) + '\n')

p3[2,3] = True
p3[4,0] = True
print('p3 = \n' + str(p3) + '\n')

print('p2 x p3 = \n' + str(p2 @ p3) + '\n')

# exactly-two-connected recursion in chained H2m
p0 = BMLP_RMS(p1.T,p2 @ p3,print_matrix=True)
print('p0 = \n' + str(p0) + '\n')

p1 = 
      0  1  2  3  4
  0|               |  0
  1|  t            |  1
  2|               |  2
  3|               |  3
  4|               |  4
      0  1  2  3  4

p2 = 
      0  1  2  3  4
  0|               |  0
  1|        t      |  1
  2|               |  2
  3|              t|  3
  4|               |  4
      0  1  2  3  4

p3 = 
      0  1  2  3  4
  0|               |  0
  1|               |  1
  2|           t   |  2
  3|               |  3
  4|  t            |  4
      0  1  2  3  4

p2 x p3 = 
      0  1  2  3  4
  0|               |  0
  1|           t   |  1
  2|               |  2
  3|  t            |  3
  4|               |  4
      0  1  2  3  4

R = R2 + I = 
      0  1  2  3  4
  0|  t            |  0
  1|     t     t   |  1
  2|        t      |  2
  3|  t        t   |  3
  4|              t|  4
      0  1  2  3  4

fixpoint = 
      0  1  2  3  4
  0|  t            |  0
  1|  t  t     t   |  1
  2|        t      |  2
  3|  t        t   |  3
  4|              t|  4


In [ ]:
# Create a sequence of boolean matrix operations to compute
#   p0(X1, X2) :- p1(X1, X3), p2(X1, X4), p3(X2, X3), p4(X2, X4), p5(X3, X4).
# => X1 - X3 
#       X  | 
# <= X2 - X4

# C1
# => X1 - X3 - X4 - X2 =>
# C2
# => X1 - X4 - X3 - X2 =>

# p1 @ p5 @ p4.T /\ C2

# x, +, ~, T, /\

num_nodes = 5
p1 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)
p2 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)
p3 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)
p4 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)
p5 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)

p1[1,3] = True
print('p1 = \n' + str(p1) + '\n')

p2[1,4] = True
print('p2 = \n' + str(p2) + '\n')

p3[2,3] = True
print('p3 = \n' + str(p3) + '\n')

p4[2,4] = True
print('p4 = \n' + str(p4) + '\n')

p5[3,4] = True
print('p5 = \n' + str(p5) + '\n')

p0 = p1 @ p5 @ p4.T * (p2 @ p5.T @ p3.T)
print('p0 = \n' + str(p0) + '\n')

p1 = 
      0  1  2  3  4
  0|               |  0
  1|           t   |  1
  2|               |  2
  3|               |  3
  4|               |  4
      0  1  2  3  4

p2 = 
      0  1  2  3  4
  0|               |  0
  1|              t|  1
  2|               |  2
  3|               |  3
  4|               |  4
      0  1  2  3  4

p3 = 
      0  1  2  3  4
  0|               |  0
  1|               |  1
  2|           t   |  2
  3|               |  3
  4|               |  4
      0  1  2  3  4

p4 = 
      0  1  2  3  4
  0|               |  0
  1|               |  1
  2|              t|  2
  3|               |  3
  4|               |  4
      0  1  2  3  4

p5 = 
      0  1  2  3  4
  0|               |  0
  1|               |  1
  2|               |  2
  3|              t|  3
  4|               |  4
      0  1  2  3  4

p0 = 
      0  1  2  3  4
  0|               |  0
  1|        t      |  1
  2|               |  2
  3|               |  3
  4|               |  4
      0  1  2  3  4

In [16]:

# Inverse a predicate using matrix transpose
def pinv(Pred):
    return Pred[0].T, 0, 0, "(" + Pred[3] + "inv)"

# Create the negation of M using sparse matrix
def pneg(Pred):
    return matrix_negate(Pred[0]), 0, 0, "(~" + Pred[3] + ")"

# Create the conjunction of two predicates if they share the same variables
def pconj(Pred1, Pred2):
    return Pred1[0] * Pred2[0], 0, 0, "(" + Pred1[3] + "/\\" + Pred2[3] + ")"

# Create the disjunction of two predicates if they share the same variables
def pdisj(Pred1, Pred2):
    return Pred1[0] + Pred2[0], 0, 0, "(" + Pred1[3] + "\/" + Pred2[3] + ")"

# Create a linear recursive predicate from base case M1 and body M2
def precur_1(Pred1, Pred2):
    return BMLP_RMS(Pred1[0], Pred2[0]), 0, 0, "(inv ->" + Pred1[3] + ", inv ->" + Pred2[3] + ", inv"  + ")"
def precur_2(Pred1, Pred2):
    return BMLP_RMS(Pred2[0], Pred1[0]), 0, 0, "(inv ->" + Pred2[3] + ", inv ->" + Pred1[3] + ", inv"  + ")"
def precur_self(Pred):
    return BMLP_RMS(Pred[0], Pred[0]), 0, 0, "(inv ->" + Pred[3] + ", inv ->" + Pred[3] + ", inv"  + ")"

# Create a chain of two predicates
def pchain_1(Pred1, Pred2):
    return Pred1[0] @ Pred2[0], 0, 0, "(inv ->" + Pred1[3] + ", " + Pred2[3] + ")"
def pchain_2(Pred1, Pred2):
    return Pred2[0] @ Pred1[0], 0, 0, "(inv ->" + Pred2[3] + ", " + Pred1[3] + ")"

# Compute the coverage of invented predicates
def pos_coverage(Pred, Pos, PN):
    return (Pred[0] * Pos).reduce_int() / PN
def neg_coverage(Pred, Neg, NN):
    return (Pred[0] * Neg).reduce_int() / NN

def apply_operators(Ms):
    newMs = []
    
    # Apply unary operators
    for pred in Ms:
        for op in unary_ops:
            newMs += [op(pred)]
    
    # Apply binary operators
    for i in range(len(Ms) - 1):
        for j in range(i + 1, len(Ms)):
            for op in binary_ops:
                fst_pred = Ms[i]
                snd_pred = Ms[j]
                newMs += [op(fst_pred, snd_pred)]
    return newMs

def init_pred(M, name):
    return (M, 0.0, 0.0, name)

unary_ops = [pinv, precur_self]
# unary_ops = [pinv, pneg, precur_self]
binary_ops = [pconj, pdisj, precur_1, precur_2, pchain_1, pchain_2] 

In [17]:
#######################################################
# Background knowledge:
# 
#			    harry+sally
#	   	        /		\
#		    john		mary
# 
# father(harry,john). mother(sally,john). 
# father(harry,mary). mother(sally,mary).
# male(harry). female(sally).
# male(john). female(mary).
# 
# Constant to matrix index mapping:
#   (0, harry), (1, john), (2, mary), (3, sally) 

# Background knowledge encoded by matrices
father = gb.Matrix.sparse(gb.BOOL, 4, 4)
father[0, 1] = True
father[0, 2] = True
father = init_pred(father, "father")

mother = gb.Matrix.sparse(gb.BOOL, 4, 4)
mother[3, 1] = True
mother[3, 2] = True
mother = init_pred(mother, "mother")

male = gb.Matrix.sparse(gb.BOOL, 4, 4)
male[0, 0] = True
male[1, 1] = True
male = init_pred(male, "male")

female = gb.Matrix.sparse(gb.BOOL, 4, 4)
female[2, 2] = True
female[3, 3] = True
female = init_pred(female, "female")

In [18]:
# Target:
#   parent
# 
# Examples:
#   E+ = 
#       {   parent(harry,john). parent(sally,john). 
#           parent(harry,mary). parent(sally,mary). }
#   E- = 
#       {   parent(harry,sally). parent(mary,john).  
#           parent(harry,harry).                    }
# Postive examples
pos = gb.Matrix.sparse(gb.BOOL, 4, 4)
pos[0, 1] = True
pos[0, 2] = True
pos[3, 1] = True
pos[3, 2] = True
PN = pos.reduce_int()
print(f'No. pos example: {PN}')

# Negative examples
neg = gb.Matrix.sparse(gb.BOOL, 4, 4)
neg[0, 3] = True
neg[2, 1] = True
neg[0, 0] = True
NN = neg.reduce_int()
print(f'No. neg example: {NN}')

depth_1 = [father, mother, male, female]
depth_2 = apply_operators(depth_1)

for ip in depth_2:
    pc = pos_coverage(ip, pos, PN)
    nc = neg_coverage(ip, neg, NN)
    if pc >= 1.0 and nc <= 0.0:
        print(ip[0])
        print(ip[3])

No. pos example: 4
No. neg example: 3
      0  1  2  3
  0|     t  t   |  0
  1|            |  1
  2|            |  2
  3|     t  t   |  3
      0  1  2  3
(father\/mother)


In [24]:
#######################################################
# Background knowledge:
# 
#			    harry+sally
#	   	        /		\
#		    john		mary
#             |          |
#           bill        maggie
# 
# father(harry,john). mother(sally,john). 
# father(harry,mary). mother(sally,mary).
# father(john, bill). mother(mary, maggie).
# male(harry). female(sally).
# male(john). female(mary).
# male(bill). female(maggie).
# 
# Constant to matrix index mapping:
#   (0, harry), (1, john), (2, mary), (3, sally), (4, bill), (5, maggie)
# 
# Target:
#   grandparent
# 
# Examples:
#   E+ = 
#       {   grandparent(harry,bill). grandparent(sally,bill). 
#           grandparent(harry,maggie). grandparent(sally,maggie). }
#   E- = 
#       {   grandparent(harry,sally). grandparent(mary,john).  
#           grandparent(harry,harry). grandparent(john, bill).  }

father[0].resize(6, 6)
mother[0].resize(6, 6)
male[0].resize(6, 6)
female[0].resize(6, 6)

father[0][1, 4] = True
mother[0][2, 5] = True
male[0][4, 4] = True
female[0][5, 5] = True

# father = init_pred(father, "father")
# mother = init_pred(mother, "mother")
# male = init_pred(male, "male")
# female = init_pred(female, "female")

# Postive examples
pos = gb.Matrix.sparse(gb.BOOL, 6, 6)
pos[0, 4] = True
pos[0, 5] = True
pos[3, 4] = True
pos[3, 5] = True
PN = pos.reduce_int()
print(f'No. pos example: {PN}')

# Negative examples
neg = gb.Matrix.sparse(gb.BOOL, 6, 6)
neg[0, 3] = True
neg[2, 1] = True
neg[0, 0] = True
neg[1, 4] = True
NN = neg.reduce_int()
print(f'No. neg example: {NN}')

depth_1 = [father, mother, male, female]
depth_2 = apply_operators(depth_1)
depth_3 = apply_operators(depth_2)
# depth_4 = apply_operators(depth_3)

for ip in depth_2:
    pc = pos_coverage(ip, pos, PN)
    nc = neg_coverage(ip, neg, NN)
    if pc >= 1.0 and nc <= 0.0:
        # print(ip[0])
        print(ip[3])

print(len(depth_2))
        
for ip in depth_3:
    pc = pos_coverage(ip, pos, PN)
    nc = neg_coverage(ip, neg, NN)
    if pc >= 1.0 and nc <= 0.0:
        # print(ip[0])
        print(ip[3])

print(len(depth_3))
        
# for ip in depth_4:
#     pc = pos_coverage(ip, pos, PN)
#     nc = neg_coverage(ip, neg, NN)
#     if pc >= 1.0 and nc <= 0.0:
#         print(ip[0])
#         print(ip[3])

redundancy = 0
for i in range(len(depth_3) - 1):
    for j in range(i + 1, len(depth_3)):
        if depth_3[i][0].iseq(depth_3[j][0]):
            redundancy += 1
            break
print(redundancy)

No. pos example: 4
No. neg example: 4
44
5764
5426
